In [24]:
import pandas as pd
import numpy as np

#Dados importados do perfil Vini0804 em 14/04/2024

diary_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\diary.csv")
likes_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\likes\films.csv")
ratings_df= pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\ratings.csv")
watched_df=pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\watched.csv")

# Manipulando os dados para juntar todos em um só dataframe 

watched_df =  watched_df.drop('Date', axis=1)  # Removendo a coluna Date pois ela não sera usada
watched_df["Like"]=watched_df["Name"].isin(likes_df["Name"]) # Criando a  coluna Like com base nos filmes que estao no dataframe Likes


#print(watched_df)

In [25]:
# Adicionando a data de quando o filme foi visto 

watched_df['First Log Date'] = np.nan  # Criando uma nova coluna 

for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_diario, filme_diario in enumerate(diary_df["Name"]):
        if filme == filme_diario:
            watched_df.loc[indice_watch, "First Log Date"] = diary_df.loc[indice_diario, "Watched Date"]
            
#print(watched_df)

In [26]:
# Adicionando o rating do filme 

watched_df['Rating'] = np.nan  # Criando uma nova coluna 

for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_rating, filme_rating in enumerate(ratings_df["Name"]):
        if filme == filme_rating:
            watched_df.loc[indice_watch, "Rating"] = ratings_df.loc[indice_rating, "Rating"]
            
#print(watched_df)

In [27]:
# Contador de filmes reassistidos

watched_df['Rewatch'] = 0
for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_diario, filme_diario in enumerate(diary_df["Name"]):
        if filme == filme_diario and diary_df["Rewatch"][indice_diario]=="Yes":
            watched_df.loc[indice_watch, 'Rewatch'] += 1

#print(watched_df)


In [28]:
 todos_os_filmes_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\TMDB_movie_dataset_v11.csv")

In [29]:
todos_os_filmes_df.columns


Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [30]:
import pandas as pd
# Colunas interessantes: revenue, runtime, budget, original_language, overview, poster_path, popularity, genres, production_companies,production_countries
# Colunas que usarei por hora: runtime, original_language, genres, production_companies,production_countries

# Supondo que "Name" seja a coluna de filmes em watched_df e "title" seja a coluna correspondente em todos_os_filmes_df

#watched_df['Duração (min)'] = pd.merge(watched_df, todos_os_filmes_df, left_on='Name', right_on='title', how='left')['runtime'] # Errado


# Convertendo a coluna 'release_date' para formato de data

todos_os_filmes_df['release_date'] = pd.to_datetime(todos_os_filmes_df['release_date'])
todos_os_filmes_df['ano'] = todos_os_filmes_df['release_date'].dt.year
watched_df = watched_df.rename(columns={'Year': 'ano', 'Name': 'title'})
watched_df['ano'] = watched_df['ano'].astype(float)



filmes_filtrados = todos_os_filmes_df[todos_os_filmes_df['title'].isin(watched_df['title'])] # Filtra os filmes que estão no dataframe watched no dataframe todos_os_filmes

filmes_df = pd.merge(watched_df, filmes_filtrados, on=['title', 'ano'], how='inner')  # Junta os dataframes contato que o nome do filme e o ano estejam juntos



            
#print(watched_df)

In [31]:
selecao_df = filmes_df[['title', 'ano','Like', 'First Log Date', 'Rating',
       'Rewatch','runtime','original_language','genres','production_companies','production_countries']]

duplicatas=selecao_df[selecao_df.duplicated(subset=['title'],keep=False)]
duplicatas 

,title,ano,Like,First Log Date,Rating,Rewatch,runtime,original_language,genres,production_companies,production_countries
8,Vice,2018.0,False,NaN,3.0,0,132,en,"Comedy, Drama","Plan B Entertainment, Gary Sanchez Productions...",United States of America
9,Vice,2018.0,False,NaN,3.0,0,13,en,Drama,NaN,NaN
23,Halloween,1978.0,False,NaN,4.0,0,91,en,"Horror, Thriller","Compass International Pictures, Falcon Interna...",United States of America
24,The Lighthouse,2019.0,True,NaN,4.5,0,109,en,"Drama, Fantasy, Thriller, Horror","A24, RT Features, New Regency Pictures, Parts ...","Brazil, Canada, United States of America"
25,The Lighthouse,2019.0,True,NaN,4.5,0,8,zh,Animation,Seesaw Creative,Macao
...,...,...,...,...,...,...,...,...,...,...,...
1606,Prey,2022.0,False,2024-02-18,NaN,0,100,en,"Thriller, Action, Science Fiction",20th Century Studios,United States of America
1607,Prey,2022.0,False,2024-02-18,NaN,0,6,en,"Horror, Drama",NaN,United States of America
1608,Prey,2022.0,False,2024-02-18,NaN,0,20,en,"Crime, Drama",Noblene,United States of America
1635,Poison,2023.0,False,2024-03-31,NaN,0,17,en,Comedy,"American Empirical Pictures, Indian Paintbrush",United States of America


In [49]:

# Linha a serem removidas  9,25,26,49,71,74,83,84, 123, 155, 177, 185,193, 194, 195, 254
duplicatas.loc[255:310]  # Continuar apartir daqui a remoção


,title,ano,Like,First Log Date,Rating,Rewatch,runtime,original_language,genres,production_companies,production_countries
264,Aladdin,1992.0,False,NaN,NaN,0,95,en,"Animation, Family, Adventure, Fantasy, Romance","Walt Disney Pictures, Walt Disney Feature Anim...",United States of America
265,Aladdin,1992.0,False,NaN,NaN,0,49,en,"Animation, Fantasy, Family, Adventure","American Film Investment Corporation, Golden F...","Japan, United States of America, Germany"
272,The Lion King,2019.0,True,NaN,NaN,0,118,en,"Adventure, Drama, Family","Walt Disney Pictures, Fairview Entertainment",United States of America
286,Aladdin,2019.0,False,NaN,NaN,0,127,en,"Adventure, Fantasy, Romance, Family","Walt Disney Pictures, Rideback",United States of America
301,Halloween,2018.0,False,NaN,4.0,0,106,en,"Horror, Thriller","Universal Pictures, Blumhouse Productions, Tra...",United States of America
304,Alice in Wonderland,2010.0,False,NaN,NaN,0,108,en,"Family, Fantasy, Adventure","Walt Disney Pictures, Roth Films, Team Todd, T...",United States of America
305,Alice in Wonderland,2010.0,False,NaN,NaN,0,99,en,NaN,NaN,NaN
309,The Jungle Book,2016.0,True,NaN,NaN,0,106,en,"Family, Adventure, Drama, Fantasy","Walt Disney Pictures, Fairview Entertainment, ...","United Kingdom, United States of America"
310,Hercules,1997.0,False,NaN,NaN,0,93,en,"Animation, Family, Fantasy, Adventure, Comedy,...","Walt Disney Pictures, Walt Disney Feature Anim...",United States of America
